In [1]:
import pymysql
import re

In [39]:
# db = pymysql.connect("127.0.0.1", "root", "", "learnqur_lqtafsir") # localhost learnqur_lqtafsir
db = pymysql.connect("94.237.67.148", "learnqur", "N!@t1khl455555", "learnqur_lqtafsir") # dev cpanel

In [40]:
c = db.cursor()

In [41]:
tafsir_name = "jalalayn" # ibnu_katsir, ringkas_kemenag, jalalayn

In [42]:
c.execute('SELECT {} FROM tafsir_id'.format(tafsir_name)) 

6236

In [43]:
with db.cursor() as c:
    # Read a single record
    sql = "SELECT `{}` FROM `tafsir_id`".format(tafsir_name)
    c.execute(sql)
    result = c.fetchall()

In [44]:
# check to ensure that you grasp the right tafsir
result[0][0]

'Al-Fatihah (Pebukaan)\r\n\r\n(Dengan nama Allah Yang Maha Pemurah lagi Maha Penyayang)'

In [8]:
db.close()

In [45]:
# db = pymysql.connect("127.0.0.1", "root", "", "learnqur_lqtafsir") # localhost db_tafsir_ringkas
db = pymysql.connect("94.237.67.148", "learnqur", "N!@t1khl455555", "learnqur_lqtafsir") # dev cpanel

In [46]:
c = db.cursor()

### Remove Duplicate Tafsir

In [11]:
# sentences = "Inilah kitab yang sempurna dan penuh keagungan, yaitu Al-Qur'an yang kami turunkan kepada nabi muhammad, tidak ada keraguan padanya tentang kebenaran apa-apa yang terkandung di dalamnya, dan orang-orang yang berakal sehat tidak akan dihinggapi keraguan bahwa Al-Qur'an berasal dari Allah karena sangat jelas kebenarannya. Alqur'an juga menjadi petunjuk yang sempurna bagi mereka yang mempersiapkan diri untuk menerima kebenaran dengan bertakwa, yaitu mengikuti segala perintah Allah dan menjauhi segala larangan-Nya agar terhindar dari siksa Allah. Meski petunjuk Al-Qur'an diperuntukkan bagi seluruh umat manusia, hanya orang-orang bertakwa saja yang siap dan mampu mengambil manfaat darinya. Inilah kitab yang sempurna dan penuh keagungan, yaitu Al-Qur'an yang kami turunkan kepada nabi muhammad, tidak ada keraguan padanya tentang kebenaran apa-apa yang terkandung di dalamnya, dan orang-orang yang berakal sehat tidak akan dihinggapi keraguan bahwa Al-Qur'an berasal dari Allah karena sangat jelas kebenarannya. Alqur'an juga menjadi petunjuk yang sempurna bagi mereka yang mempersiapkan diri untuk menerima kebenaran dengan bertakwa, yaitu mengikuti segala perintah Allah dan menjauhi segala larangan-Nya agar terhindar dari siksa Allah. Meski petunjuk Al-Qur'an diperuntukkan bagi seluruh umat manusia, hanya orang-orang bertakwa saja yang siap dan mampu mengambil manfaat darinya."
# sentences_processed = make_it_distinct(sentences)

def make_it_distinct(sentences):
    words_array = sentences.split(". ")
    for words in words_array:
        count = 0
        #print("-- words: ", words)
        for words_idx in words_array:
            #print("-- words_idx: ", words_idx)
            if words in words_idx:
                count+=1
                #print("counted!")
                if count > 1:
                    #print("found!")
                    seen = set()
                    words_distinct = [x for x in sentences.split(".") if x not in seen and not seen.add(x)]
                    words_distinct.pop(-1) # remove ""
                    words_distinct.pop(-1) # remove last duplicate
                    words_joint = ". ".join(words_distinct)

                    return words_joint
                    break
        
        return sentences            
        break

In [12]:
def remove_line_spacing(sentences):
    # remove multiple line breaks
    text_processed = '\r\n\r\n'.join([x for x in sentences.splitlines() if x.strip()])
    text_processed = text_processed.split('\r\n\r\n')
    text_processed = ' '.join(text_processed)
    
    return text_processed

### Set DB into DB

In [13]:
def preprocessDoc(text):
       
    # remove multiple whitespaces
    text_processed = re.sub(' +',' ',text)
    
    # capitalize first letter of every sentence (should not be used for ibnu katsir & jalalayn)
    # text_processed = '. '.join(map(lambda s: s.strip().capitalize(), text_processed.split('.')))
    
    # print(text_processed)
    
    # preprocess specific typos    
    text_processed = text_processed.replace("allah", "Allah")
    text_processed = text_processed.replace("muhammad", "Muhammad")
    text_processed = text_processed.replace("Al-Qur?an", "Al-Qur'an")
    text_processed = text_processed.replace("al-Qur?an", "Al-Qur'an")
    text_processed = text_processed.replace("al-qur?an", "Al-Qur'an")
    text_processed = text_processed.replace("al-qur'an", "Al-Qur'an")
    text_processed = text_processed.replace("AlQur'an", "Al-Qur'an")
    text_processed = text_processed.replace("Alqur'an", "Al-Qur'an")
    
    # preprocess apostrophe symbol
    text_no_apostrophe = text_processed.replace("'", "\\'")

    text_processed = text_no_apostrophe.replace(".", ". ")
    text_processed = text_processed.replace("?", "? ")
    text_processed = text_processed.replace(",", ", ")
    text_processed = text_processed.replace("kejadiankejadian", "kejadian-kejadian")
    text_processed = text_processed.replace("hurufhuruf", "huruf-huruf")
    text_processed = text_processed.replace("kitabkitab", "kitab-kitab")
    text_processed = text_processed.replace("Najr? n", "Najran")
    text_processed = text_processed.replace("? Arsy", "Arsy")
    text_processed = text_processed.replace("? ", "\\'")
    text_processed = text_processed.replace("\\'am\\'d", "Samud")
    text_processed = text_processed.replace("Mahf\\'z", "Mahfuz")
    text_processed = text_processed.replace("berimandialah", "beriman. Dialah")
    text_processed = text_processed.replace("makhlukdialah", "makhluk. Dialah")
    text_processed = text_processed.replace(".Dialah", ". Dialah")
    text_processed = text_processed.replace("-nya", "-Nya")
    text_processed = text_processed.replace("-mu", "-Mu")
    text_processed = text_processed.replace("an-nisa\'\'/4: 69", "an-nisa 4:69")
    text_processed = text_processed.replace("''/4: 69", " 4:69") 
    text_processed = text_processed.replace("-Mualif", "-Mu. Alif") 
    
    text_processed = text_processed.replace("swt", "Subhanahu wa ta\\'ala")
    text_processed = text_processed.replace("saw", "Shallallahu \\'alaihi wa sallam")
    
    text_processed = text_processed.replace("Swt", "Subhanahu wa ta\\'ala")
    text_processed = text_processed.replace("Saw", "Shallallahu \\'alaihi wa sallam")
    
    text_processed = text_processed.replace("swt.", "Subhanahu wa ta\\'ala")
    text_processed = text_processed.replace("saw.", "Shallallahu \\'alaihi wa sallam")
    
    text_processed = text_processed.replace("Swt.", "Subhanahu wa ta\\'ala")
    text_processed = text_processed.replace("Saw.", "Shallallahu \\'alaihi wa sallam")
    
    text_processed = text_processed.replace("swt:", "Subhanahu wa ta\\'ala")
    text_processed = text_processed.replace("saw:", "Shallallahu \\'alaihi wa sallam")
    
    text_processed = text_processed.replace("Swt:", "Subhanahu wa ta\\'ala:")
    text_processed = text_processed.replace("Saw:", "Shallallahu \\'alaihi wa sallam:")
    
    # remove multiple whitespaces should not be used for ibnu katsir & jalalayn)
    # text_processed = re.sub(' +',' ',text_processed)
    
    # remove multiple line breaks should not be used for ibnu katsir & jalalayn)
    # text_processed = '\r\n\r\n'.join([x for x in text_processed.splitlines() if x.strip()])
    
    # remove multiple full stop
    text_processed = text_processed.replace(". .", ".") 
    
    # remove redundant ayah should not be used for ibnu katsir & jalalayn)
    # text_processed = make_it_distinct(text_processed)
    
    # remove line spacing (should not be used for ibnu katsir & jalalayn)
    # text_processed = remove_line_spacing(text_processed)
     
    # print(text_processed)
    
    return text_processed

In [14]:
len(result)

6236

In [15]:
# check to ensure that you grasp the right tafsir
result[0][0]

''

In [16]:
row_arr = []
for idx, row in enumerate(result): # result[0:20]
    print("%d..." % idx)

    # add dot at the end of paragraph
    if row[0][len(row[0])-1] is not '.' and row[0][len(row[0])-1] is not ' ':
        # print(row[0][len(row[0])-1])
        row_arr.append(row[0] + ".")
    else:
        #print(row[0][len(row[0])-1])
        row_arr.append(row[0])

    # print(row_arr[idx])
    row_text_processed = preprocessDoc(row_arr[idx])
  
    query = "UPDATE tafsir_id SET {} ='".format(tafsir_name) + row_text_processed + "' WHERE id = " + str(idx+1) # tafsir_id changes quran_id_complete, tafsir_ringkas changes ringkas_kemenag
    c.execute(query)

0...


IndexError: string index out of range

In [ ]:
# row_arr = []
# for idx, row in enumerate(result): # result[0:20]
#     print("%d..." % idx)

#     if((idx+1) != 3188 and (idx+1) != 3189 and (idx+1) != 3190 and (idx+1) != 3266 and 
#        (idx+1) != 3267 and (idx+1) != 3270 and (idx+1) != 3271 and (idx+1) != 4625 and 
#        (idx+1) != 4626 and (idx+1) != 4809 and (idx+1) != 4810 and (idx+1) != 5187 and 
#        (idx+1) != 5188 and (idx+1) != 5249 and (idx+1) != 5250 and (idx+1) != 5298 and 
#        (idx+1) != 5299 and (idx+1) != 5300 and (idx+1) != 5331 and (idx+1) != 5332 and 
#        (idx+1) != 5447 and (idx+1) != 5448):
        
#         # add dot at the end of paragraph
#         if row[0][len(row[0])-1] is not '.' and row[0][len(row[0])-1] is not ' ':
#             # print(row[0][len(row[0])-1])
#             row_arr.append(row[0] + ".")
#         else:
#             #print(row[0][len(row[0])-1])
#             row_arr.append(row[0])
             
#         # print(row_arr[idx])
#         row_text_processed = preprocessDoc(row_arr[idx])
#     else:
                
#         # add dot at the end of paragraph
#         if row[0][len(row[0])-1] is not '.':
#             row_arr.append(row[0] + ".")
#         else:
#             row_arr.append(row[0])
            
#         row_no_apostrophe = row[0]
#         row_text_processed = row_no_apostrophe
    
#     query = "UPDATE tafsir_id SET ringkas_kemenag ='" + row_text_processed + "' WHERE id = " + str(idx+1) # tafsir_id changes quran_id_complete, tafsir_ringkas changes ringkas_kemenag
#     c.execute(query)

In [ ]:
db.commit()

### Reset all tafsir text (optional)

In [ ]:
for idx, row in enumerate(result):    
    pass
    # print(idx)
    # query = "UPDATE tafsir_id SET ringkas_kemenag ='' WHERE id = " + str(idx+1) # tafsir_id changes quran_id_complete
    # c.execute(query)

In [ ]:
db.commit()

In [ ]:
db.close()

Todo:

1. Remove duplicate line spacing
2. Remove duplicate words 

<hr/>